# **Data Cleaning**

## Objectives

* Prepare the dataset for ML by:
  * Assessing and handling missing data.
  * Removing irrelevant or problematic features.
  * Ensuring consistency in data types.
  * Saving clean datasets for training and testing.

## Inputs

* `outputs/datasets/cleaned/df_main_for_cleaning.csv`: output from the EDA notebook, ready for cleaning.

## Outputs

* `outputs/datasets/cleaned/TrainSetCleaned.csv`: cleaned training dataset.
* `outputs/datasets/cleaned/TestSetCleaned.csv`: cleaned test dataset.

## Additional Comments

* This notebook focuses solely on the cleaned house records (`df_main`).  
* `inherited_houses.csv` is kept separate and will be used later for prediction once the model is trained.  

---

# Change Working Directory

In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Collected data

In [ ]:
import pandas as pd

# Load the dataset prepared during EDA
file_path = "outputs/datasets/cleaned/df_main_for_cleaning.csv"
df = pd.read_csv(file_path)
df.head(5)

---

# Data Exploration

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute / len(df) * 100, 2)

    df_missing_data = (
        pd.DataFrame({
            "RowsWithMissingData": missing_data_absolute,
            "PercentageOfDataset": missing_data_percentage,
            "DataType": df.dtypes
        })
        .sort_values(by='PercentageOfDataset', ascending=False)
        .query("PercentageOfDataset > 0")
    )

    return df_missing_data

EvaluateMissingData(df)

In [ ]:
from ydata_profiling import ProfileReport

vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("✅ There are no variables with missing data")

# Correlation and PPS Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, mask=mask, cmap='viridis',
                    annot_kws={"size": font_annot}, ax=ax, linewidth=0.5)
        ax.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()

def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, mask=mask, cmap='rocket_r',
                    annot_kws={"size": font_annot}, ax=ax, linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()

def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman", numeric_only=True)
    df_corr_pearson = df.corr(method="pearson", numeric_only=True)

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')

    return df_corr_pearson, df_corr_spearman, pps_matrix

def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold):
    print("*** Spearman Correlation ***")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=(12, 10), font_annot=10)

    print("*** Pearson Correlation ***")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=(12, 10), font_annot=10)

    print("*** Power Predictive Score (PPS) ***")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=(12, 10), font_annot=10)

df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)
DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold=0.4, PPS_Threshold=0.2)

# Data Cleaning

Strategy to Drop Variables (if any)
We drop columns if:
* More than 80% missing values
* Duplicated or irrelevant (e.g., unique IDs)

In [ ]:
from feature_engine.selection import DropFeatures

features_to_drop = [] # depending on previous output

print(f"* {len(features_to_drop)} variables to drop \n\n{features_to_drop}")

dropper = DropFeatures(features_to_drop=features_to_drop)
df = dropper.fit_transform(df)

Impute Missing Values

In [ ]:
from sklearn.impute import SimpleImputer

categorical_cols = df.select_dtypes(include='object').columns
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns

# Fill categorical with mode
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

# Fill numerical with median
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

# SPlit Train and Test Set

In [ ]:
from sklearn.model_selection import train_test_split

TrainSet, TestSet = train_test_split(df, test_size=0.2, random_state=42)

print(f"✅ TrainSet shape: {TrainSet.shape}")
print(f"✅ TestSet shape: {TestSet.shape}")

EvaluateMissingData(TrainSet)

---

# Save Cleaned Data

In [ ]:
import os
try:
    os.makedirs("outputs/datasets/cleaned", exist_ok=True)
except Exception as e:
    print(e)

TrainSet.to_csv("outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)
TestSet.to_csv("outputs/datasets/cleaned/TestSetCleaned.csv", index=False)


---

# Data Cleaning Summary

* Imputed all missing numerical values using median
* Imputed categorical columns with most frequent value
* Saved TrainSetCleaned and TestSetCleaned